In [163]:
import numpy as np
import matplotlib as plt
import pandas as pd

# class Node deals with the information about the relation and information that each node contains 
class Node:
    def __init__(self, name, major, year, reg_day, priority):
        self.name = name # student's name
        self.major = major # major of the student
        self.year = year # the year the student is in
        self.reg_day = reg_day # day of registration 
        self.priority = priority # used in comparsion based analysis for developing priority 

        # when inputing a node there is no child attached 
        self.left = None 
        self.right = None
        self.height = 1 # will be used to help determine rebalancing 

# class AVL_Tree contains information about the structure of AVL_Tree and how to maintain it
# such as the rotation, insertion, and deletion form the tree
class AVL_Tree:
    def __init__(self):
        self.root = None # initially, the tree is empty because we have not put in any new students 

    # determines the tree height of each subtree, helps to determine if the subtree needs to be rotated and in which direction
    def tree_height(self, node):
        if node is None:
            return 0
        left_height = self.tree_height(node.left) # getting the height of the left subtree
        right_height = self.tree_height(node.right) # getting the height of the right subtree
        return max(left_height, right_height) + 1 # return the subtree which has the largest depth (adding 1 since we start our count at 0)
    
    def left_left_rotation(self, node):
        if node is not None and node.right is not None:
            new_parent = node.right # setting the right child as the pivot/new parent
            temp_var = new_parent.left # temparary variable that will be reattached when tree has been rotated
            new_parent.left = node # setting parent node as left child
            node.right = temp_var # since the node is going to be smaller than the temp var, it is going to be placed as right child

            # since we repositioned the parent node and the right child's position, we need to update the height of each
            node.height = max(self.tree_height(node.left), self.tree_height(node.right)) + 1 
            new_parent.height = max(self.tree_height(new_parent.left), self.tree_height(new_parent.right)) + 1

            # return the new_parent for recalculation to see if the tree is now balanced
            return new_parent
        else:
            return node
        
    def right_right_rotation(self, node):
        if node is not None and node.left is not None:
            new_parent = node.left # setting the left child as the pivot/new parent
            temp_var = new_parent.right # temparary variable that will be reattached when tree has been rotated
            new_parent.right = node # setting parent node as right child
            node.left = temp_var # since the node is going to be bigger than the temp var, it is going to be placed as left child

            # since we repositioned the parent node and the right child's position, we need to update the height of each
            node.height = max(self.tree_height(node.left), self.tree_height(node.right)) + 1
            new_parent.height = max(self.tree_height(new_parent.left), self.tree_height(new_parent.right)) + 1

            # return the new_parent for recalculation to see if the tree is now balanced
            return new_parent
        else:
            return node

    def left_right_rotation(self, node):
        node.left = self.left_left_rotation(node.left)
        return self.right_right_rotation(node)

    def right_left_rotation(self, node):
        node.right = self.right_right_rotation(node.right)
        return self.left_left_rotation(node)
 
    # inserting a new student into the AVL tree
    def insert(self, parent_node, name, major, year, reg_day, priority):
        # set the student to the root node if it is going to be ther first thing in the list 
        if parent_node is None:
            # in here we care using the node class to create the root node
            return Node(name, major, year, reg_day, priority)  

        # We are inserting the following students based on their priority value
        # the new student has a lower priority value than the students already in the tree
        if priority < parent_node.priority:
            parent_node.left = self.insert(parent_node.left, name, major, year, reg_day, priority)
            
        # the new student has a higher priority value than the students already in the tree
        else:
            parent_node.right = self.insert(parent_node.right, name, major, year, reg_day, priority)

        # once we add the new node we need to update the height of our tree that will be used to determine 
        # balance of our new tree
        parent_node.height = max(self.tree_height(parent_node.left), self.tree_height(parent_node.right)) + 1
       
        # if the balance is greater than 1 or smaller than -1 (so 2 or -2) means we are no longer balanced 
        # thus we need to rotate the nodes to rebalance the tree such that it is between -1>= x =<1
        balance = self.tree_height(parent_node.left) - self.tree_height(parent_node.right)
        
        # there is a string of left child that outweighs how many right child there is in the subtree
        if balance > 1:
            if priority < parent_node.left.priority:
                return self.right_right_rotation(parent_node)
            else:
                return self.left_right_rotation(parent_node)
            
        # there is a string of right child that outweighs how many left child there is in the subtree
        if balance < -1:
            if priority > parent_node.right.priority:
                return self.left_left_rotation(parent_node)
            else:
                return self.right_left_rotation(parent_node)

        return parent_node


class stud_registration:
    def __init__(self):
        self.registraion_list = AVL_Tree() # prioritization structure of students
        self.student_dict = {}  # Dictionary to store student information
        
    # updates the student_dict when a new student is added to the tree
    def update_student_dict(self):
        self.student_dict = {}  # Clear the dictionary
        self._update_student_dict(self.registraion_list.root) # update dictionary based on new student

    # this is different because _update_student is intended for internal use in relation to update_student_dict
    # it should only be accessed within the classs it exists 
    def _update_student_dict(self, node):
        if node:
            # the set up of:
            # node.right
            # self.studnt_dict[node.name]
            # node.left
            # ensures that the dictionary will be ordered in a descending manner
            self._update_student_dict(node.right) 

            # Update the student_dict with the student's information
            self.student_dict[node.name] = {
                'major': node.major,
                'year': node.year,
                'reg_day': node.reg_day,
                'priority': node.priority
            }

            self._update_student_dict(node.left)

    def student_priority(self, major, year, reg_day):
        level = 0 #reset the level for each function call
        if year == 0: #for auditors
            level = level + (.5-(.1*(reg_day))) # level will equal 0.5 for registrations before orientation week, 0.4 for registrations on the first day, etc.
          
        elif year <5: # for undergraduate students
            if major == 'cs' or major == 'math':
                level = level+10 # adds 10 to the level for cs or math majors,
            level = level + year # adds 1,2,3, or 4 to the level based on the year.
            level = level +(.5-(.1*(reg_day))) #+0.5 for students registered before the orientation week, same as for auditors.
            
        else:
            level = 15 #for graduate students, level will automatically be 15.
        return level

    # adding new student to the AVL tree so long as they registered within the orientation period
    def register_student(self, name, year, reg_day, major):
        if int(reg_day) < 6:
            priority = self.student_priority(major, year, reg_day) # caluclte their priority
            self.registraion_list.root = self.registraion_list.insert(self.registraion_list.root, name, major, year, reg_day, priority)
            self.update_student_dict()  # Update the student_dict
       
    # we are removing students based on the dictionary list. I am not updating tree because
    # I am running under the assumption that the class has already started and we already of a waitlist which is captured 
    # in the dictionary
    def remove_student(self, name):
        if name in self.student_dict:
            del self.student_dict[name] # delete item when key matches name

    # # returns the dictionary of students
    # def get_student_dict(self):
    #     return self.student_dict
    
    def get_student_dataframe(self, max_students=25):
        admitted_students = [] # building dataframe to visually represent students who are admided
        count = 0 # only go through the top 25 in dictionary
        for name, student_info in self.student_dict.items():
            # adding student to dictionary
            admitted_students.append([name, student_info['major'], 
                                 student_info['year'], 
                                 student_info['reg_day'], 
                                 student_info['priority']])
            count += 1

            # When there are 25 admitted students, we stop 
            if count >= max_students:
                break

        # getting column for each data frame
        columns = ['Name', 'Major', 'Year', 'Reg Day', 'Priority']

        # return dataframe 
        return pd.DataFrame(admitted_students, columns=columns)



if __name__ == "__main__":
    reg_stud = stud_registration()

    # Insert student
    reg_stud.register_student('student1',2,0,'cs')
    reg_stud.register_student('student2',5,0,'data science')
    reg_stud.register_student('student3',3,2,'math')
    reg_stud.register_student('student4',0,5,'none')
    reg_stud.register_student('student5',1,4,'science')
    reg_stud.register_student('student6',2,2,'art')
    reg_stud.register_student('student7',6,5,'data science')
    reg_stud.register_student('student8',3,5,'math')
    reg_stud.register_student('student9',7,3,'data science')
    reg_stud.register_student('student10',0,3,'none')
    reg_stud.register_student('student11',1,0,'cs')
    reg_stud.register_student('student12',4,3,'math')
    reg_stud.register_student('student13',4,0,'physics')
    reg_stud.register_student('student14',3,5,'cs')
    reg_stud.register_student('student15',2,5,'math')
    reg_stud.register_student('student16',0,0,'none')
    reg_stud.register_student('student17',4,4,'science')
    reg_stud.register_student('student18',5,4,'data science')
    reg_stud.register_student('student19',1,0,'math')
    reg_stud.register_student('student20',1,2,'science')
    reg_stud.register_student('student21',2,4,'cs')
    reg_stud.register_student('student22',1,2,'math')
    reg_stud.register_student('student23',3,3,'history')
    reg_stud.register_student('student24',7,0,'data science')
    reg_stud.register_student('student25',4,0,'cs')
    reg_stud.register_student('student26',3,0,'math')
    reg_stud.register_student('student27',2,4,'cs')
    reg_stud.register_student('student28',2,3,'art')
    reg_stud.register_student('student29',1,0,'science')
    reg_stud.register_student('student30',1,2,'science')
    reg_stud.register_student('student31',3,0,'cs')
    reg_stud.register_student('student32',4,0,'math')
    reg_stud.register_student('student33',2,4,'science')


    # delete student
    reg_stud.remove_student('student18')
    reg_stud.remove_student('student24')

    #return admitted students as a data frame
    admitted = reg_stud.get_student_dataframe()
    print(admitted)
   


         Name         Major  Year  Reg Day  Priority
0    student9  data science     7        3      15.0
1    student7  data science     6        5      15.0
2    student2  data science     5        0      15.0
3   student32          math     4        0      14.5
4   student25            cs     4        0      14.5
5   student12          math     4        3      14.2
6   student31            cs     3        0      13.5
7   student26          math     3        0      13.5
8    student3          math     3        2      13.3
9   student14            cs     3        5      13.0
10   student8          math     3        5      13.0
11   student1            cs     2        0      12.5
12  student27            cs     2        4      12.1
13  student21            cs     2        4      12.1
14  student15          math     2        5      12.0
15  student19          math     1        0      11.5
16  student11            cs     1        0      11.5
17  student22          math     1        2    

In [152]:
reg_stud.register_student('student1',2,0,'cs')
reg_stud.register_student('student2',5,0,'data science')
reg_stud.register_student('student3',3,2,'math')
reg_stud.register_student('student4',0,5,'none')
reg_stud.register_student('student5',1,4,'science')
reg_stud.register_student('student6',2,2,'art')
reg_stud.register_student('student7',6,5,'data science')
reg_stud.register_student('student8',3,5,'math')
reg_stud.register_student('student9',7,3,'data science')
reg_stud.register_student('student10',0,3,'none')
reg_stud.register_student('student11',1,0,'cs')
reg_stud.register_student('student12',4,3,'math')
reg_stud.register_student('student13',4,0,'physics')
reg_stud.register_student('student14',3,5,'cs')
reg_stud.register_student('student15',2,5,'math')
reg_stud.register_student('student16',0,0,'none')
reg_stud.register_student('student17',4,4,'science')
reg_stud.register_student('student18',5,4,'data science')
reg_stud.register_student('student19',1,0,'math')
reg_stud.register_student('student20',1,2,'science')
reg_stud.register_student('student21',2,4,'cs')
reg_stud.register_student('student22',1,2,'math')
reg_stud.register_student('student23',3,3,'history')
reg_stud.register_student('student24',7,0,'data science')
reg_stud.register_student('student25',4,0,'cs')
reg_stud.register_student('student26',3,0,'math')
reg_stud.register_student('student27',2,4,'cs')
reg_stud.register_student('student28',2,3,'art')
reg_stud.register_student('student29',1,0,'science')
reg_stud.register_student('student30',1,2,'science')
reg_stud.register_student('student31',3,0,'cs')
reg_stud.register_student('student32',4,0,'math')
reg_stud.register_student('student33',2,4,'science')


reg_stud.remove_student('student33')

students = reg_stud.get_student_dataframe()
print(students)

AttributeError: 'NoneType' object has no attribute 'right'